In [1]:
import pygame
import random
from PJ_03_bird import Bird
from PJ_03_pipe import Pipe

pygame 2.5.2 (SDL 2.28.3, Python 3.8.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def generate_pipes(last_pipe_time, pipe_frequency, pipe_sprite):
    now = pygame.time.get_ticks()
    if now - last_pipe_time >= pipe_frequency:
        random_height = random.randint(-100, 100)
        pipe_btm = Pipe(SCREEN_WIDTH, SCREEN_HEUGHT*0.5+pipe_gap+random_height, pipe_img, False)
        pipe_top = Pipe(SCREEN_WIDTH, SCREEN_HEUGHT*0.5-pipe_gap+random_height, flip_pipe_img, True)
        pipe_sprite.add(pipe_btm)
        pipe_sprite.add(pipe_top)
        return now
    return last_pipe_time

def draw_score():
    score_text = score_font.render(str(score), True, WHITE)
    window.blit(score_text, (SCREEN_WIDTH*0.5-score_text.get_width()*0.5, 20))

In [4]:
pygame.init()
FPS=60
SCREEN_WIDTH=780
SCREEN_HEUGHT=600

window= pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEUGHT))
pygame.display.set_caption('Flappy bird')
clock= pygame.time.Clock()

#導入圖片
bg_img= pygame.image.load('./PJ_03_Flappy bird/bg.png')
bg_img= pygame.transform.scale(bg_img, (SCREEN_WIDTH, SCREEN_HEUGHT))
# bird_img= pygame.image.load('./PJ_03_Flappy bird/bird1.png')
bird_imgs = []
for i in range(1, 3):
    bird_imgs.append(pygame.image.load(f'./PJ_03_Flappy bird/bird{i}.png'))
    
ground_img= pygame.image.load('./PJ_03_Flappy bird/ground.png')
restart_img= pygame.image.load('./PJ_03_Flappy bird/restart.png')
pipe_img= pygame.image.load('./PJ_03_Flappy bird/pipe.png')
flip_pipe_img = pygame.transform.flip(pipe_img, False, True)

pygame.display.set_icon(bird_imgs[0])

# 載入字體
score_font = pygame.font.Font('./PJ_03_Flappy bird/微軟正黑體.ttf', 60)

# 遊戲變數
ground_speed= 4
ground_x= 0
pipe_gap= 75
pipe_frequency = 1500
last_pipe_time = pygame.time.get_ticks() - pipe_frequency
ground_top = SCREEN_HEUGHT-100
game_over = False
score = 0
WHITE = (255, 255, 255)

# sprite群組
bird = Bird(100, SCREEN_HEUGHT*0.5, bird_imgs)
bird_sprite = pygame.sprite.Group()
bird_sprite.add(bird)

pipe_sprite = pygame.sprite.Group()

run=True
while run:
    clock.tick(FPS)
    # 取得輸入
    for event in pygame.event.get():
        if event.type==pygame.QUIT:
            run=False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            if event.button==1 and not game_over:
                bird.jump()
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE and game_over:
                game_over = False
                score = 0
                last_pipe_time = pygame.time.get_ticks() - pipe_frequency
                bird.reset()
                pipe_sprite.empty()
                
    # 更新遊戲
    bird_sprite.update(ground_top)
    if not game_over:
        pipe_sprite.update()
        last_pipe_time = generate_pipes(last_pipe_time, pipe_frequency, pipe_sprite)
        #取得管子位置
        first_pipe = pipe_sprite.sprites()[0]
        #判斷是否通過得分
        if not first_pipe.bird_pass:
            if first_pipe.rect.right <  bird.rect.left:
                score += 1
                first_pipe.bird_pass = True
        # 移動地板
        ground_x -= ground_speed
        if ground_x < -100:
            ground_x=0

    # 碰撞判斷
    if pygame.sprite.groupcollide(bird_sprite, pipe_sprite, False, False) or\
        bird.rect.top <=0 or\
        bird.rect.bottom >= ground_top:
        game_over = True
        bird.game_over()
    else:
        pass

    # 畫面顯示
    window.blit(bg_img, (0, 0))
    bird_sprite.draw(window)
    pipe_sprite.draw(window)
    # window.blit(bird_img, (100, 100))
    # window.blit(pipe_img, (300, 400))
    window.blit(ground_img, (ground_x, (ground_top)))
    draw_score()
    if game_over:
        window.blit(restart_img, (SCREEN_WIDTH*0.5-restart_img.get_width()*0.5, SCREEN_HEUGHT*0.5-restart_img.get_height()*0.5))
    pygame.display.update()

pygame.quit()